In [1]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import pandas as pd

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# read IEA data    
file_path = "inputs/WEO2023_AnnexA_Free_Dataset_Regions.csv"
outlook = pd.read_csv(file_path)
outlook.drop('PUBLICATION', axis=1, inplace=True)
outlook = outlook.set_index(['SCENARIO', 'CATEGORY', 'PRODUCT', 'FLOW', 'UNIT', 'REGION', 'YEAR'])
outlook = outlook.loc[outlook.index.get_level_values('FLOW') == 'Electricity generation']
outlook_product = pd.pivot_table(outlook, values='VALUE', index=['SCENARIO', 'REGION', 'YEAR'], columns='PRODUCT', aggfunc='sum')

In [3]:
#start completing the missing outlook to match the electricity product in EXIOBASE
outlook_product["Other Renewables"] = outlook_product["Renewables"] -  (outlook_product["Solar PV"] + outlook_product["Wind"])
outlook_product["Others"] = outlook_product["Total"] - (outlook_product["Coal"] + outlook_product["Natural gas"] + outlook_product["Nuclear"] + outlook_product["Renewables"])

In [4]:
#List variables that will be usefull during parsing
elec_sectors = exio3.Z.filter(like="Electricity", axis=0).index.get_level_values(1).unique().to_list()
renewables_list = ['Electricity by hydro','Electricity by wind','Electricity by biomass and waste','Electricity by solar photovoltaic','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal']
other_renewables_list = ['Electricity by hydro','Electricity by biomass and waste','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal']
oilothers_sectors = ["Electricity by petroleum and other oil derivatives","Electricity nec"]
high_emission_sectors = ["Electricity by petroleum and other oil derivatives","Electricity by coal", "Electricity by gas"]
low_emission_sectors = ['Electricity by nuclear','Electricity by hydro','Electricity by wind','Electricity by biomass and waste','Electricity by solar photovoltaic','Electricity by solar thermal','Electricity by tide, wave, ocean','Electricity by Geothermal',"Electricity nec"]

In [5]:
region_list = list(dict.fromkeys(outlook_product.index.get_level_values("REGION")))
standard_names = coco.convert(names=region_list, to='EXIO3')
country_map = pd.DataFrame({   
    "region": region_list,
    "exio3": standard_names
})
country_map.set_index(['region'],inplace=True)

In [6]:
#setting the available direct values (coal, gas, nuclear, wind, solar) from outlook
outlook_product[elec_sectors]=0
outlook_product["Electricity by coal"]=outlook_product["Coal"]
outlook_product["Electricity by gas"]=outlook_product["Natural gas"]
outlook_product["Electricity by nuclear"]=outlook_product["Nuclear"]
outlook_product["Electricity by wind"]=outlook_product["Wind"]
outlook_product["Electricity by solar photovoltaic"]=outlook_product["Solar PV"]

In [7]:
x_elec = exio3.x.filter(like="Electricity", axis=0)

#distributing outlook of other renewables based values from outlook from specified regionals
x_elec_other_renewables = x_elec[x_elec.index.get_level_values('sector').isin(other_renewables_list)]
x_elec_other_renewables_total = x_elec_other_renewables.groupby(level=0).sum()
x_elec_other_renewables_normalized = x_elec_other_renewables.div(x_elec_other_renewables_total)

#distributing outlook of other electricity (oil and nec) based values from outlook from specified regionals
x_elec_oilothers = x_elec[x_elec.index.get_level_values('sector').isin(oilothers_sectors)]
x_elec_oilothers_total = x_elec_oilothers.groupby(level=0).sum()
x_elec_oilothers_normalized = x_elec_oilothers.div(x_elec_oilothers_total)

for index, row in outlook_product.iterrows():
    source = country_map.loc[index[1]]["exio3"]
    if source!= "not found":
        for sector in other_renewables_list: # for other renewables
            proportion = x_elec_other_renewables_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Other Renewables"] * proportion
        for sector in oilothers_sectors: # for oil and others/nec 
            proportion = x_elec_oilothers_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Others"] * proportion

In [8]:
# FIRST round aggregation
region_agg = coco.agg_conc(original_countries='EXIO3', 
                           aggregates=["EU",
                                       {"ZA" : "Africa",
                                        "WF" : "Africa",
                                        "MX" : "Central and South America",
                                        "WL" : "Central and South America",
                                        "BR" : "Central and South America",
                                        "CA" : "North America",
                                        "US" : "North America",
                                        "WA" : "Asia Pacific",
                                        "AU" : "Asia Pacific",
                                        "JP" : "Asia Pacific",
                                        "CN" : "Asia Pacific",
                                        "KR" : "Asia Pacific",
                                        "IN" : "Asia Pacific",
                                        "TW" : "Asia Pacific",
                                        "ID" : "Asia Pacific",
                                        "WM" : "Middle East"
                                       }],
                           missing_countries=True, 
                           merge_multiple_string=None)

exio3agg = exio3.aggregate(region_agg = region_agg,inplace=False)
exio3agg.calc_all()

In [9]:
country_map["exio3"] = "not found"
country_map.loc["Africa"]["exio3"] = "Africa"
country_map.loc["Asia Pacific"]["exio3"] = "Asia Pacific"
country_map.loc["Central and South America"]["exio3"] = "Central and South America"
country_map.loc["European Union"]["exio3"] = "EU"
country_map.loc["Middle East"]["exio3"] = "Middle East"
country_map.loc["North America"]["exio3"] = "North America"

In [10]:
x_elec = exio3agg.x.filter(like="Electricity", axis=0) # using new aggregated regions

#distributing outlook of other renewables based values from outlook from specified regionals
x_elec_other_renewables = x_elec[x_elec.index.get_level_values('sector').isin(other_renewables_list)]
x_elec_other_renewables_total = x_elec_other_renewables.groupby(level=0).sum()
x_elec_other_renewables_normalized = x_elec_other_renewables.div(x_elec_other_renewables_total)

#distributing outlook of other electricity (oil and nec) based values from outlook from specified regionals
x_elec_oilothers = x_elec[x_elec.index.get_level_values('sector').isin(oilothers_sectors)]
x_elec_oilothers_total = x_elec_oilothers.groupby(level=0).sum()
x_elec_oilothers_normalized = x_elec_oilothers.div(x_elec_oilothers_total)

for index, row in outlook_product.iterrows():
    source = country_map.loc[index[1]]["exio3"]
    if source!= "not found":
        for sector in other_renewables_list: # for other renewables
            proportion = x_elec_other_renewables_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Other Renewables"] * proportion
        for sector in oilothers_sectors: # for oil and others/nec 
            proportion = x_elec_oilothers_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Others"] * proportion

In [11]:
# SECOND round aggregation
region_agg = coco.agg_conc(original_countries='EXIO3', 
                           aggregates=[{"ID" : "Southeast Asia"},"continent"],
                           missing_countries=True, 
                           merge_multiple_string=None)

exio3agg = exio3.aggregate(region_agg = region_agg,inplace=False)
exio3agg.calc_all()

In [12]:
country_map["exio3"] = "not found"
country_map.loc["Europe"]["exio3"] = "Europe"
country_map.loc["Southeast Asia"]["exio3"] = "Southeast Asia"

In [13]:
x_elec = exio3agg.x.filter(like="Electricity", axis=0) # using new aggregated regions

#distributing outlook of other renewables based values from outlook from specified regionals
x_elec_other_renewables = x_elec[x_elec.index.get_level_values('sector').isin(other_renewables_list)]
x_elec_other_renewables_total = x_elec_other_renewables.groupby(level=0).sum()
x_elec_other_renewables_normalized = x_elec_other_renewables.div(x_elec_other_renewables_total)

#distributing outlook of other electricity (oil and nec) based values from outlook from specified regionals
x_elec_oilothers = x_elec[x_elec.index.get_level_values('sector').isin(oilothers_sectors)]
x_elec_oilothers_total = x_elec_oilothers.groupby(level=0).sum()
x_elec_oilothers_normalized = x_elec_oilothers.div(x_elec_oilothers_total)

for index, row in outlook_product.iterrows():
    source = country_map.loc[index[1]]["exio3"]
    if source!= "not found":
        for sector in other_renewables_list: # for other renewables
            proportion = x_elec_other_renewables_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Other Renewables"] * proportion
        for sector in oilothers_sectors: # for oil and others/nec 
            proportion = x_elec_oilothers_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Others"] * proportion

In [14]:
# THIRD round aggregation
region_agg = coco.agg_conc(original_countries='EXIO3', 
                           aggregates=[],
                           missing_countries="World", 
                           merge_multiple_string=None)

exio3agg = exio3.aggregate(region_agg = region_agg,inplace=False)
exio3agg.calc_all()

In [15]:
country_map["exio3"] = "not found"
country_map.loc["World"]["exio3"] = "World"

In [16]:
x_elec = exio3agg.x.filter(like="Electricity", axis=0) # using new aggregated regions

#distributing outlook of other renewables based values from outlook from specified regionals
x_elec_other_renewables = x_elec[x_elec.index.get_level_values('sector').isin(other_renewables_list)]
x_elec_other_renewables_total = x_elec_other_renewables.groupby(level=0).sum()
x_elec_other_renewables_normalized = x_elec_other_renewables.div(x_elec_other_renewables_total)

#distributing outlook of other electricity (oil and nec) based values from outlook from specified regionals
x_elec_oilothers = x_elec[x_elec.index.get_level_values('sector').isin(oilothers_sectors)]
x_elec_oilothers_total = x_elec_oilothers.groupby(level=0).sum()
x_elec_oilothers_normalized = x_elec_oilothers.div(x_elec_oilothers_total)

for index, row in outlook_product.iterrows():
    source = country_map.loc[index[1]]["exio3"]
    if source!= "not found":
        for sector in other_renewables_list: # for other renewables
            proportion = x_elec_other_renewables_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Other Renewables"] * proportion
        for sector in oilothers_sectors: # for oil and others/nec 
            proportion = x_elec_oilothers_normalized.loc[source,sector]["indout"]
            outlook_product.loc[index,sector] = row["Others"] * proportion

In [17]:
outlook_product = outlook_product[outlook_product.index.get_level_values('REGION') != 'Eurasia']

In [18]:
outlook_product[elec_sectors].to_csv("data/outlook_emix_product.csv")

In [19]:
# SECOND PART OF PARSING 
# creating ratio or standardized version of outlook
file_path = "data/outlook_emix_product.csv"
outlook_product = pd.read_csv(file_path)

In [20]:
outlook_STD = outlook_product.set_index(["SCENARIO","REGION","YEAR"],inplace=False)
outlook_STD_total = pd.DataFrame(index = outlook_STD.index, columns = outlook_STD.columns)
outlook_STD_sum = outlook_STD.sum(axis=1)

for index,value in outlook_STD_sum.items():
    outlook_STD_total.loc[index] = value

outlook_STD = outlook_STD.div(outlook_STD_total)

In [21]:
outlook_STD.to_csv("data/outlook_emix_STD.csv")

In [22]:
# THRID PART OF PARSING 
# creating detailed outlook per region per sector that match the A structure, but only for electricity 
scenario_list = ["Announced Pledges Scenario","Stated Policies Scenario"]
scenario_year_list = [2030,2050]

In [23]:
def build_outlook_scenario(scenario, year):
    outlook_scenario = outlook_all[(outlook_all.SCENARIO == scenario) & (outlook_all.YEAR == year)].drop(columns =['SCENARIO','YEAR'])
    outlook_scenario = pd.melt(outlook_scenario, id_vars=['Exio3'], var_name='Product', value_name='Value')
    outlook_scenario.set_index(['Exio3', 'Product'], inplace=True)
    outlook_scenario.index.names = ['region', 'sector']
    outlook_scenario = outlook_scenario.reindex(Z_elec.index)
    #get the energy mix of actual usage from EXIO intermediary
    Z_elec_total = Z_elec.groupby(level=0).sum()
    Z_elec_STD = Z_elec.div(Z_elec_total)
    Z_elec_STD.fillna(0,inplace=True)

    outlook = pd.DataFrame(index = Z_elec_STD.index, columns = Z_elec_STD.columns)
    for col in outlook.columns:
        outlook[col] = outlook_scenario['Value']
    
    Z_elec_STD_total = Z_elec_STD.groupby(level=0).sum()
    outlook = outlook.mul(Z_elec_STD_total)

    outlook_gap = outlook - Z_elec_STD
    outlook_gap.loc[(slice(None), low_emission_sectors),:] = 0 # focus on high emission sectors
    # ignore outlook  where actual data is lower than the target, it means the target/outlook has been achieved. The remaining gap need to be subsidezed by other sectors
    outlook_gap = pd.DataFrame(np.where(outlook_gap < 0, 0, outlook_gap), index=outlook_gap.index, columns=outlook_gap.columns) 

    outlook_low = outlook.copy()
    outlook_low.loc[(slice(None), high_emission_sectors),:] = 0
    outlook_low_total = outlook_low.groupby(level=0).sum()
    outlook_low_STD = outlook_low.div(outlook_low_total)
    outlook_low_STD.fillna(0,inplace=True)

    outlook_gap_total = outlook_gap.groupby(level=0).sum()
    outlook_gap_subs = outlook_low_STD.mul(outlook_gap_total) # get the amount of subsidy burdened by each sector not part of high emission sectors
    outlook_gap_subs = outlook_gap_subs-outlook_gap # the amount of final subsidy, including negative subsidy for outlook of high emission sectors

    new_outlook = outlook + outlook_gap_subs

    filename = "data/outlook_emix_"+scenario.replace(" ", "")+"_"+str(year)+".csv" 
    new_outlook.to_csv(filename)
    return

In [24]:
file_path = "inputs/input_outlook_country_mapping.csv"
outlook_mapping = pd.read_csv(file_path)

file_path = "data/outlook_emix_STD.csv"
outlook_STD = pd.read_csv(file_path)

In [25]:
Z_elec = exio3.Z.filter(like="Electricity", axis=0)
outlook_all = outlook_mapping.merge(outlook_STD, left_on='OutlookIEA', right_on='REGION',how = "outer")
outlook_all.drop(columns =['OutlookIEA','REGION'], inplace=True)

In [26]:
for scenario in scenario_list: 
    for year in scenario_year_list:
        build_outlook_scenario(scenario,year)